# Tai resume va role dataset tu kaggle ve va luu vao HDFS

In [ ]:

# TODO: Lưu datasetDict IT_role vào HDFS dưới dạng file raw 
import os
from datasets import load_dataset, concatenate_datasets
import pandas as pd # Dùng pandas để hỗ trợ kiểm tra và thao tác (tùy chọn)

# 1. Tải Dataset
IT_role = load_dataset("NxtGenIntern/IT_Job_Roles_Skills_Certifications_Dataset")

print(f"✅ Đã tải DatasetDict với các phần: {list(IT_role.keys())}")
print("-" * 30)

# 2. Kết hợp toàn bộ các phần (splits) thành một Dataset duy nhất
# print("➡️ Đang tiến hành kết hợp các phần (train, validation, test) thành 1 Dataset...")

# Lấy danh sách các Dataset object (ví dụ: [IT_role['train'], IT_role['validation']])
# all_datasets = list(IT_role.values())

# Sử dụng hàm concatenate_datasets để kết hợp chúng
# combined_dataset = concatenate_datasets(all_datasets)

combined_dataset = IT_role['train']

print(f"🎉 Đã kết hợp thành công. Tổng số hàng: {len(combined_dataset)}")
print("-" * 30)

# 3. Định nghĩa đường dẫn và tên file Parquet
OUTPUT_DIR = "resume_and_role"
OUTPUT_FILENAME = "IT_role.parquet"
OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)

# Tạo thư mục nếu nó chưa tồn tại
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

print(f"Tạo thư mục output cục bộ: {OUTPUT_DIR}")
print("-" * 30)

# 4. Lưu Dataset đã kết hợp thành MỘT file Parquet
# Sử dụng phương thức to_parquet() với tham số writer_batch_size=None 
# (hoặc đảm bảo nó không tạo ra nhiều file)
# Tuy nhiên, cách đơn giản nhất là chuyển sang Pandas DataFrame rồi lưu
try:
    # Chuyển đổi Dataset sang Pandas DataFrame
    df_combined = combined_dataset.to_pandas()
    
    # Lưu Pandas DataFrame sang MỘT file Parquet
    df_combined.to_parquet(OUTPUT_PATH, index=False)
    
    print(f"💾 Đã lưu toàn bộ dữ liệu vào MỘT file Parquet duy nhất tại: {OUTPUT_PATH}")
    
except Exception as e:
    print(f"Lỗi khi lưu bằng Pandas/PyArrow: {e}")
    
# TODO: Đẩy file parquet lên HDFS

In [5]:

from huggingface_hub import hf_hub_download
import pandas as pd
import os

REPO_ID = "datasetmaster/resumes"
FILENAME = "master_resumes.jsonl"

# Chỉ định thư mục đích rõ ràng
DEST_DIR = "./resume_and_role"

file_path = hf_hub_download(
    repo_id=REPO_ID,
    filename=FILENAME,
    repo_type="dataset",
    local_dir=DEST_DIR,                # thư mục đích
    local_dir_use_symlinks=False       # đảm bảo copy file thật, không chỉ symlink
)

print("File đã tải về:", file_path)

os.rename(file_path, os.path.join(DEST_DIR, "IT_resume.jsonl"))
# IT_resume_df = pd.read_json(
#     hf_hub_download(repo_id=REPO_ID, filename=FILENAME, repo_type="dataset"),
#     lines = True
# )

d:\code language\anaconda3\envs\bigdata\Lib\site-packages\huggingface_hub\utils\_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(


File đã tải về: resume_and_role\master_resumes.jsonl


# Tai cac du lieu can thiet tu HDFS

TODO: Thay the cac duong dan den cac file tuong ung

In [ ]:
from pyspark.sql import SparkSession

# 1. Khởi tạo Spark Session (Bắt buộc)
# Thêm config dfs.client.use.datanode.hostname để fix lỗi kết nối DataNode
spark = SparkSession.builder \
    .appName("Doc Data Tu HDFS") \
    .config("spark.hadoop.dfs.client.use.datanode.hostname", "true") \
    .getOrCreate()

# 2. Đường dẫn HDFS (Sửa lại host/port cho đúng với setup của bạn)
# Nếu chạy từ local máy tính kết nối vào Docker: dùng localhost hoặc host.docker.internal

# Hoặc nếu chạy production/internal network:
# hdfs_path = "hdfs://namenode:9000/path/to/file.jsonl"

# các bảng job
itviec_path = "hdfs://host.docker.internal:9000/path/to/file.jsonl" 
careerviet_path = "hdfs://host.docker.internal:9000/path/to/file.jsonl" 
vietnamwork_path = "hdfs://host.docker.internal:9000/path/to/file.jsonl" 
topcv_path = "hdfs://host.docker.internal:9000/path/to/file.jsonl" 

# bảng resume
resume_path = "hdfs://host.docker.internal:9000/path/to/IT_resume.jsonl"


# bảng skill
role_path = "hdfs://host.docker.internal:9000/path/to/IT_role.parquet"


# 3. Đọc file
# Do Spark hoạt động theo cơ chế Lazy Evaluation, dòng này chỉ mới check metadata
df_itviec = spark.read.json(itviec_path)
df_CareerViet = spark.read.json(careerviet_path)
df_VietnamWork = spark.read.json(vietnamwork_path)
df_TopCV = spark.read.json(topcv_path)

df_resume = spark.read.json(resume_path)
df_role = spark.read.parquet(role_path)

## Tach bang company tu df_itviec

In [ ]:
from pyspark.sql.functions import col
df_company_detail = df_itviec.select(
    
    
    # Trích xuất các trường con từ 'company_detail'
    col("company_detail.company_name").alias("main_company_name"),
    col("company_detail.declaration"),
    col("company_detail.`Company type`").alias("company_type"), # Dùng dấu `` cho tên cột có khoảng trắng
    col("company_detail.`Company industry`").alias("company_industry"),
    col("company_detail.`Company size`").alias("company_size"),
    col("company_detail.Country"),
    col("company_detail.`Working days`").alias("working_days"),
    col("company_detail.`Overtime policy`").alias("overtime_policy")
    
).distinct()

# Xóa trường company khỏi df_itviec để tránh trùng lặp khi join
df_itviec = df_itviec.drop("company_detail")

# Gộp các bảng job với nhau thành bảng lớn theo union

Sử dụng union chay để gọp mà không cần biết tên cột như thế nào
Điều tiên quyết phải là thứ tự các cột phải giống hệt nhau

In [ ]:
# Sử dụng union chay để gọp mà không cần biết tên cột như thế nào
# Điều tiên quyết phải là thứ tự các cột phải giống hệt nhau

from functools import reduce

list_df_jobs = [df_itviec, df_CareerViet, df_VietnamWork, df_TopCV]

df_job = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), list_df_jobs)




# Join broadcast với bảng company và job

In [ ]:
# trước đó có thể cấu hình broadcast cực nhỏ để đám bảo chỉ 1 bảng được broadcast
# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 1024 )  # 1KB


from pyspark.sql.functions import broadcast

df_job_company = df_job.join(
    # Sử dụng hàm broadcast() trên DataFrame nhỏ hơn
    broadcast(df_company_detail), 
    df_job["company_name"] == df_company_detail["main_company_name"],
    "inner"
)

# TODO: Lưu df_job_company này lên cache


# TODO:Lưu df_job_company này lên cache

# Join bảng df_job_company với df_role theo key vị trí công việc (role - job title)
TODO: có thể sử dụng chính bảng df_job_company đã được cache ở trên

In [ ]:
# Join bảng df_job_company với df_role theo key vị trí công việc (role - job title)

# bắt buộc config để đảm bảo sort merge join thay vì broadcast
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 1024 )  # 1KB

df_job_company_role = df_job_company.join(
    df_role,
    df_job_company['Role']== df_role['Job Title'],
    'inner'
)




# Multiple joins với df_resume theo key tên ứng viên (name)

In [ ]:
# Multiple joins với df_resume theo key tên ứng viên (name)


# Nên sử dụng CBO cho multiple join

# Để sử dụng CBO thì chỉ cần config trong Spark Session và thống kê các bảng trước (vì CBO chỉ làm việc với các bảng đã có thống kê)


# 1. KHỞI TẠO SPARK SESSION VÀ BẬT CBO
# Cấu hình CBO và các tham số liên quan
spark.conf.set("spark.sql.cbo.enabled", "true")
spark.conf.set("spark.sql.cbo.joinReorder.enabled", "true")
spark.conf.set("spark.sql.statistics.fallBackToHadoopFileLength", "false")

# Đảm bảo CBO được bật
print(f"CBO Enabled: {spark.conf.get('spark.sql.cbo.enabled')}")

# --- DỮ LIỆU GIẢ LẬP ---


# 2. ĐĂNG KÝ BẢNG TẠM VÀ THU THẬP THỐNG KÊ (ANALYZE)
# CBO chỉ hoạt động trên các bảng đã có thống kê.
df_job.createOrReplaceTempView("job")
df_role.createOrReplaceTempView("role")
df_company_detail.createOrReplaceTempView("company")

print("\n--- Bắt đầu thu thập Thống kê ---")
# Thu thập thống kê cho tất cả các bảng và cột
spark.sql("ANALYZE TABLE job COMPUTE STATISTICS FOR ALL COLUMNS").show()
spark.sql("ANALYZE TABLE role COMPUTE STATISTICS FOR ALL COLUMNS").show()
spark.sql("ANALYZE TABLE company COMPUTE STATISTICS FOR ALL COLUMNS").show()
print("--- Kết thúc thu thập Thống kê ---")

# 3. THỰC HIỆN TRUY VẤN VỚI NHIỀU PHÉP JOIN (3 BẢNG)
# Yêu cầu Spark tự động xác định thứ tự join tối ưu:
sql_query = """
SELECT 
    t1.ten, 
    t2.ten_ky_nang, 
    t3.cap_bac
FROM 
    job t1
INNER JOIN 
    role t2
ON 
    t1.role = t2['Job Title']
INNER JOIN
    company t3
ON 
    t1.company_name = t3.main_company_name
"""

df_job_company_role = spark.sql(sql_query)

# 4. KIỂM TRA KẾ HOẠCH THỰC THI
print("\n--- KẾ HOẠCH THỰC THI (Execution Plan) được tối ưu bởi CBO ---")
# CBO sẽ quyết định thứ tự join và loại join (BroadcastHashJoin, SortMergeJoin)
# dựa trên kích thước và thống kê đã thu thập được.
df_job_company_role.explain(extended=True)
